In [ ]:
import json
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import boto3
#boto3 is a package created by aws,programatically communicate with aws services
#lambda doesn't support all of the functions that we equired, we need to use lambda layer instead of 'pip install ---'
from datetime import datetime
def lambda_handler(event, context):
    client_id = os.environ.get('client_id')
    client_secret = os.environ.get('client_secret')
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)
    playlists = sp.user_playlists('spotify')
    playlist_link = "https://open.spotify.com/playlist/37i9dQZF1DWTt3gMo0DLxA"
    playlist_URI = playlist_link.split("/")[-1]
    data = sp.playlist_tracks(playlist_URI)
    print(data)

    client = boto3.client('s3')
    filename = "spotify_raw_" + str(datetime.now()) + ".json"
    client.put_object(
        Bucket ="spotify-etl-project-srinivas",
        Key = "raw_data/toprocessed/" + filename,
        Body = json.dumps(data)
    )


    glue= boto3.client("glue")
    gluejobname = "spotify_transformation_job1"

    try:
        runId = glue.start_job_run(JobName=gluejobname)
        status= glue.get_job_run(JobName = gluejobname, RunId = runId['JobRunId'])
        print("Job Status : ",status['JobRun']['JobRunState'])
    except Exception as e:
        print(e)



